<a href="https://colab.research.google.com/github/vishal-burman/PyTorch-Architectures/blob/master/misc/T5_Augment(Synthetic_Data).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# An experiment notebook for creating synthetic data specifically for T5 finetuning

In [ ]:
! pip install datasets
! pip install transformers
! pip install nlpaug

In [62]:
from datasets import load_dataset
import nltk
from nltk import word_tokenize, sent_tokenize
import spacy
import nlpaug.augmenter.word as naw
from random import randrange

nltk.download('punkt')
nlp = spacy.load("en_core_web_sm") # Feels more robust than NLTK...will use this!

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
dataset = load_dataset("squad")

In [6]:
contexts = []
for sample in dataset["train"]:
  if len(contexts) >= 1000:
    break
  
  contexts.append(sample["context"])
print(f"Length --> {len(contexts)}")

Length --> 1000


In [26]:
contexts = sorted(contexts, key=len, reverse=True)

On April 4, 2008, Beyoncé married Jay Z. She publicly revealed their marriage in a video montage at the listening party for her third studio album, I Am... Sasha Fierce, in Manhattan's Sony Club on October 22, 2008. I Am... Sasha Fierce was released on November 18, 2008 in the United States. The album formally introduces Beyoncé's alter ego Sasha Fierce, conceived during the making of her 2003 single "Crazy in Love", selling 482,000 copies in its first week, debuting atop the Billboard 200, and giving Beyoncé her third consecutive number-one album in the US. The album featured the number-one song "Single Ladies (Put a Ring on It)" and the top-five songs "If I Were a Boy" and "Halo". Achieving the accomplishment of becoming her longest-running Hot 100 single in her career, "Halo"'s success in the US helped Beyoncé attain more top-ten singles on the list than any other woman during the 2000s. It also included the successful "Sweet Dreams", and singles "Diva", "Ego", "Broken-Hearted Girl"

In [68]:
aug_insert = naw.ContextualWordEmbsAug(model_path="bert-base-uncased", action="insert")
aug_replace = naw.ContextualWordEmbsAug(model_path="bert-base-uncased", action="substitute")
aug_backtranslation = naw.BackTranslationAug(
    from_model_name='facebook/wmt19-en-de', 
    to_model_name='facebook/wmt19-de-en'
)

Downloading:   0%|          | 0.00/825 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.00G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/825 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.00G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/67.0 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/308k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/67.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/829k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/829k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/308k [00:00<?, ?B/s]

In [71]:
def insert_or_replace_word(text: str):
  """
  Part 1
  """
  doc = nlp(text)
  augmented_texts = []
  prob = randrange(2)
  for sent in list(doc.sents):

    if prob == 0:
      augmented_text = aug_insert.augment(str(sent))
    else:
      augmented_text = aug_replace.augment(str(sent))

    augmented_texts.append(augmented_text)
  final_text = " ".join(augmented_texts)
  return final_text

def backtranslate_sentence(text: str):
  """
  Part 2
  """
  final_text = aug_backtranslation.augment(text)
  if final_text == text:
    return None
  return final_text

In [70]:
some_text = contexts[len(contexts) - 4]
print(f"Original text --> {some_text}")
# print(insert_or_replace_word(some_text))
print(backtranslate_sentence(some_text))

Original text --> On February 6, 2016, one day before her performance at the Super Bowl, Beyoncé released a new single exclusively on music streaming service Tidal called "Formation".
On February 6, 2016, the day before her Super Bowl performance, Beyoncé released a new single called "Formation" exclusively on the music streaming service Tidal.
